In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime
import os
os.chdir('/home/reffert/DeepAR_InfluenzaForecast')
import PythonFiles.model
from PythonFiles.Configuration import Configuration
from PythonFiles.HpTuning import get_data, objectiveDeepAR
from ray.air import session
from ray import tune
data_splits_dict = get_data(truncate=False, with_features=True)[0]

In [ ]:
hp_search_space = {
    "num_cells": tune.grid_search([80, 140]),
    "num_layers": tune.grid_search([6, 8, 10]),
    "context_length":tune.grid_search([1, 2, 104]),
    "cell_type":tune.grid_search(["lstm"]),
    "epochs":tune.grid_search([140, 160, 200]),
    "use_feat_static_real":tune.grid_search([False]),
    "use_feat_dynamic_real":tune.grid_search([True]),
    "use_feat_static_cat":tune.grid_search([False]),
    "cardinality":tune.sample_from(lambda spec:[2]*411 if spec.config.use_feat_static_cat else None),
}


train = data_splits_dict["with_features_2001"][0]
test = data_splits_dict["with_features_2001"][1]
configuration = Configuration()

tuner = tune.Tuner(
    tune.with_parameters(objectiveDeepAR, train=train, test=test, configuration=configuration),
    tune_config=tune.TuneConfig(
        num_samples = 5,
        metric="mean_WIS",
        mode="min",
        max_concurrent_trials=24,
    ),
    param_space=hp_search_space,
)
results = tuner.fit()

print("Best hyperparameters found were: ", results.get_best_result().config)

results_df = results.get_dataframe()
print(results_df)
results_df.to_csv("Hyperparameter_results_03_06.csv")

2023-06-03 10:20:49,400	INFO worker.py:1553 -- Started a local Ray instance.


100%|██████████| 50/50 [00:11<00:00,  4.31it/s, epoch=85/200, avg_epoch_loss=0.638]
(objectiveDeepAR pid=1959539) 
 80%|████████  | 40/50 [00:51<00:12,  1.28s/it, epoch=31/160, avg_epoch_loss=0.535]
